# IA para espacialização dos dados

In [24]:
import os
import rasterio
import numpy as np
import pandas as pd
import asciichartpy
import geopandas as gpd
import rioxarray as rxr

from tqdm import tqdm, trange
from pyproj import Transformer
from utils.consts import SOIL_TYPES
from rasterio.transform import rowcol
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split

### Device disponível para treinar o modelo MLP

In [2]:
# 1) Dispositivo (GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


### Dataset com os dados para convolução
---
- O Dataset nesta aplicação é extremamente importante pois irá englobar todas as informações necessárias para espacializar os dado de infiltração

In [ ]:
# 2) Dataset
class MeuDataset(Dataset):
    nix_bands = [
        "R400 nm", "R410 nm", "R420 nm", "R430 nm",
        "R440 nm", "R450 nm", "R460 nm", "R470 nm",
        "R480 nm", "R490 nm", "R500 nm", "R510 nm",
        "R520 nm", "R530 nm", "R540 nm", "R550 nm",
        "R560 nm", "R570 nm", "R580 nm", "R590 nm",
        "R600 nm", "R610 nm", "R620 nm", "R630 nm",
        "R640 nm", "R650 nm", "R660 nm", "R670 nm",
        "R680 nm", "R690 nm", "R700 nm",
    ]

    columns_dados = ["Ponto", "Lat", "Lon", "soils_type", "Clay", "Silt", "Sand", "K (C1)"]

    def __init__(self, device:torch.device|None=None):
        """
        O dataset tem o formato de uma tupla com os valores em X e em Y:
        - (X, Y)
        - Onde:
        - X: (rasters_vals, [Dist. Talvegue, Tipo do Solo, na ordem de soil_types])
        - Y: K (cm/s)
        """
        self.device = device
        self.janela = 25 # Janela deve ser ímpar

        if self.janela%2 == 0:
            raise ValueError("A janela deve ser ímpar")

        # Lendo Tabelas
        print("Lendo Tabelas")
        self.dados = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="Infiltracao")
        self.dados = self.dados[self.columns_dados].dropna().reset_index(drop=True)
        gdf = gpd.GeoDataFrame(self.dados, geometry=gpd.points_from_xy(self.dados["Lat"], self.dados["Lon"]), crs="EPSG:4326")
        gdf.to_crs("EPSG:31983", inplace=True)
        self.dados['Lat'] = gdf.geometry.y
        self.dados['Lon'] = gdf.geometry.x

        self.nix = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="Nix")
        self.pXRF = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="pXRF")

        # Lendo Rasteres importantes
        self.talvegues = gpd.read_file(r"D:/Mestrado/Trabalho Final/SIG/HidrografiaArea.zip")

        # Lendo Rasteres
        print("Lendo Rasteres")
        self.uso_solo        = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/USOSOLO.tif")                 # Tipos de uso do solo
        self.elevation       = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Elevation.tif")               # Elevação
        self.terrain_rug_idx = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/TerrainRuggednessIndex.tif")  # Variação de elevação entre um pixel e seus vizinhos imediatos
        self.topo_pos_idx    = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/TopograficPositionIndex.tif") # Elevação de um ponto com a média da elevação ao redor, topo, vale ou plano
        self.roughness       = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Roughness.tif")               # A diferença entre a elevação máxima e mínima dentro de uma vizinhança
        self.slope           = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Slope.tif")                   # Declividade
        self.aspect          = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Aspect.tif")                  # Para onde "aponta" a face do terreno
        self.texture_02      = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\textura_2.tif")               # Textura a 2 cm
        self.texture_20      = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\textura_20.tif")              # Textura a 20 cm

        # Dados dos rasteres concatenados
        self.raster_data = np.array([
            self.uso_solo.values[0],
            self.elevation.values[0],
            self.terrain_rug_idx.values[0],
            self.topo_pos_idx.values[0],
            self.roughness.values[0],
            self.slope.values[0],
            self.aspect.values[0],
            self.texture_02.values[0],
            self.texture_20.values[0],
        ])

        self.transformer = Transformer.from_crs("EPSG:31983", self.uso_solo.rio.crs, always_xy=True)
        self.transform = self.uso_solo.rio.transform()

        self._process_dados()

    def _process_dados(self):
        # Dados para convolução
        x, y = self.transformer.transform(self.dados["Lon"], self.dados["Lat"])
        row, col = rowcol(self.transform, x, y)

        jan = int((self.janela-1)/2)

        start_row = row-jan
        end_row   = row+jan

        start_col = col-jan
        end_col   = col+jan

        self.dados['s_row']=start_row
        self.dados['e_row']=end_row
        self.dados['s_col']=start_col
        self.dados['e_col']=end_col

        # Distância até o talvegue principal
        linhas_unidas = self.talvegues.union_all()
        pontos = gpd.GeoSeries(gpd.points_from_xy(self.dados["Lon"], self.dados["Lat"]), crs="EPSG:31983")
        dists = pontos.apply(lambda p: p.distance(linhas_unidas))
        self.dados["dist_talvegue"] = dists


    def __len__(self):
        return len(self.dados)

    def __getitem__(self, i):
        idx = i
        if isinstance(i, (int, float)):
            idx = [i]

        # Pontos
        pontos = self.dados.loc[idx]

        # Valores nos rasteres
        rasters_vals = []
        for s_row, e_row, s_col, e_col in zip(pontos['s_row'], pontos['e_row'], pontos['s_col'], pontos['e_col']):
            rasters_vals.append(self.raster_data[:, s_row:e_row+1, s_col:e_col+1])
        
        rasters_vals = torch.tensor(np.array(rasters_vals), device=self.device, dtype=torch.float64)

        # Demais dados
        K = torch.tensor(pontos["K (C1)"].values, device=self.device) * 1000 # K*1000 pois os valores estão baixos de mais
        dist_talvegue = torch.tensor(pontos[["dist_talvegue"]].values, device=self.device)

        if isinstance(i, (int, float)):
            dist_talvegue = dist_talvegue[0]
            rasters_vals = rasters_vals[0]
            K = K[0]


        return (rasters_vals, dist_talvegue), K
    
dataset = MeuDataset(device=device)

print("Len:", len(dataset))
dataset[15][0][0].shape, dataset[1:3][0][0].shape

Lendo Tabelas
Lendo Rasteres
Len: 83


(torch.Size([9, 25, 25]), torch.Size([3, 9, 25, 25]))

### Configurações do treino
---

- Seed para números aleatórios
- % de treino e teste
- Métricas
- Epochs
- Batch Size

In [ ]:
# Seed para permitir reprodutibilidade dos valores pseudo-aleatórios
seed = 42

# Porcentagens de Treino e Teste
train_percent = 80
test_percent  = 20

# BatchSize e Epochs
batch_size = 2
epochs     = 1000  # Poucos pontos, verificar overfitting

In [7]:
n = len(dataset)
n_train = int(train_percent*n/100)
n_test = n - n_train
g = torch.Generator().manual_seed(seed)

train_ds, test_ds = random_split(dataset, [n_train, n_test], generator=g)

print("N Total:", n, "N Train:", n_train, "N Teste:", n_test)

train_ds[1:5][0][0].shape, test_ds[1:5][0][0].shape

N Total: 83 N Train: 70 N Teste: 13


(torch.Size([4, 9, 25, 25]), torch.Size([4, 9, 25, 25]))

### MLP e CNN
---

MLP configurada com uma CNN também

In [9]:
# Modelo (MLP e CNN)
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels=9,    # Número de bandas do raster de entrada
                out_channels=16,
                kernel_size=3,
                padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        ).to(dtype=torch.float64)
        self.flatten = nn.Flatten().to(dtype=torch.float64)
        self.fc = nn.Sequential(
            nn.Linear(
                (1152) + 1, # Tamanho da saída da convolução + dist_talvegue
                64
            ),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.GELU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.Sigmoid(),
            nn.Linear(16, 1)
        ).to(dtype=torch.float64)

    def forward(self, raster, values):
        x = self.conv(raster)
        x = self.flatten(x)
        x = torch.cat([x, values], dim=1)
        x = self.fc(x)
        return x

mlp = MLP().to(device=device)

X = dataset[1:4][0]
mlp(*X)

tensor([[0.1069],
        [0.1090],
        [0.1053],
        [0.1080]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddmmBackward0>)

### Ajustar o modelo MLP

- Processos para ajustar a MLP pelo método do gradiente descendente

In [32]:
def nse(y_pred, y_true, mean=None):
    if mean is not None:
        y_true_mean = mean
    else:
        y_true_mean = torch.mean(y_true)
        
    numerator = torch.sum((y_pred - y_true) ** 2)
    denominator = torch.sum((y_true - y_true_mean) ** 2)
    return 1 - (numerator / denominator)

In [35]:
os.makedirs("best_model/", exist_ok=True)

# Média dos valores para compara erros
mean = dataset[:][1].mean()

model = MLP().to(device=device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = CyclicLR(
    optimizer,
    base_lr=0.0001,      # menor LR
    max_lr=0.1,          # maior LR
    step_size_up=100,    # número de iterações até atingir o max_lr
    mode='triangular',   # ou 'triangular2', 'exp_range'
    cycle_momentum=True  # necessário se usar Adam em vez de SGD
)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=batch_size)

best_test_loss = torch.inf
best_state = None

train_losses = []
val_losses = []

for epoch in range(1, epochs+1):
    # --- treino ---
    model.train()
    running_loss = 0.0
    for x, y in train_loader:

        optimizer.zero_grad()
        y_sim = model(*x)

        loss = criterion(y_sim[:, 0], y)
        running_loss += loss.item() * x[1].size(0)

        loss.backward()
        optimizer.step()
    
    train_loss = running_loss / len(train_loader.dataset) # type: ignore
    train_losses.append(train_loss)

    # --- validação ---
    model.eval()
    val_loss = 0.0
    nash = 0.0

    with torch.no_grad():
        for x, y in test_loader:
            # Forward
            y_sim = model(*x)

            loss = criterion(y_sim[:, 0], y)
            val_loss += loss.item() * x[1].size(0)

            ns = nse(y_sim[:, 0], y, mean)
            nash += ns.item() * x[1].size(0)

    # Média do loss
    val_loss /= len(test_loader.dataset) # type: ignore
    nash /= len(test_loader.dataset)     # type: ignore
    val_losses.append(val_loss)

    # early stopping simples
    current_lr = optimizer.param_groups[0]['lr']
    if val_loss < best_test_loss:
        best_test_loss = val_loss
        best_state = {
            "lr":current_lr,
            "epoch": epoch,
            "epochs":epochs,
            "val_loss":val_loss,
            "train_loss":train_loss,
            "batch_size":batch_size,
            "model_state":model.state_dict(),
        }

        torch.save(best_state, f"best_model/{str(val_loss).replace(".", "_")}.pth")

    # Altero a taxa de aprendizado a cada fim da epoch para melhorar o aprendizado
    scheduler.step()

    #Print das métricas atuais

    indices = np.linspace(0, len(val_losses) - 1, 150, dtype=int)
    subset = [val_losses[i] for i in indices]

    ascii_chart = asciichartpy.plot(subset, {'height': 15})

    # Limpo o terminal
    clear_output(wait=True)
    print(f"Epoch {epoch}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Kc NASH: {(nash):.6f}cm/s | At lr:{current_lr:.4e}\n{ascii_chart}")


Epoch 1000/1000 | Train Loss: 95.1472 | Val Loss: 152.2005 | Kc NASH: 0.093644cm/s | At lr:1.0990e-03
  231.77  ┼╮
  226.27  ┤│
  220.77  ┤│
  215.27  ┤│
  209.78  ┤│
  204.28  ┤│
  198.78  ┤│
  193.28  ┤│
  187.79  ┤│
  182.29  ┤│
  176.79  ┤│
  171.29  ┤╰╮                                        ╭╮                                ╭╮
  165.80  ┤ │         ╭╮  ╭╮                       ╭╮││     ╭╮                     ╭╮  ││                                                     ╭╮
  160.30  ┤ │       ╭╮││  ││ ╭─╮    ╭╮             │╰╯╰╮╭╮ ╭╯│              ╭╮     ││  ││                 ╭╮ ╭╮ ╭─╮ ╭─╮╭╮                ╭╮   ││          ╭╮
  154.80  ┤ ╰─────╮╭╯╰╯╰╮╭╯╰─╯ ╰────╯╰─────────╮╭──╯   │││ │ │╭─────────────╯╰──╮╭─╯│  │╰╮╭───────────────╯╰─╯╰─╯ ╰╮│ ╰╯╰────────────────╯╰──╮│╰─╮╭╮╭╮╭───╯╰────
  149.30  ┤       ╰╯    ╰╯                     ╰╯      ╰╯╰─╯ ╰╯                 ╰╯  ╰──╯ ╰╯                        ╰╯                        ╰╯  ╰╯╰╯╰╯
  143.81  ┤
